In [1]:
import os
from google.colab import drive

# 1. 드라이브 연결
drive.mount('/content/drive')

# 2. 파일이 있는 폴더로 이동 (이미지의 경로 기준)
os.chdir('/content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016')

print(f"현재 위치: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
현재 위치: /content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split  # 추가됨
from tqdm import tqdm

# 1. 설정
# ==============================================================================
# 'subject': 사람 기준 분리 (일반화 성능 평가용 - 어려움)
# 'random':  전체 섞어서 분리 (학습 잘 됐는지 확인용 - 쉬움, 점수 높게 나옴)
SPLIT_METHOD = 'random'
# ==============================================================================

# 2. PKL 로드
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/HAR_data/RealWorld2016/RealWorld2016_Master_Sync.pkl")

WINDOW_SIZE = 100
STEP = 50
FEATURE_COLS = [c for c in df.columns if 'acc_' in c or 'gyr_' in c]

# 3. 라벨 인코딩
le = LabelEncoder()
df['label'] = le.fit_transform(df['activity'])
print("Classes:", le.classes_)

# 4. Windowing 함수
def create_dataset(df, window_size, step):
    Xs, ys = [], []
    grouped = df.groupby(['subject_id', 'activity', 'trial'])

    for _, group in grouped:
        data = group[FEATURE_COLS].values
        label = group['label'].values[0]

        for i in range(0, len(data) - window_size, step):
            window = data[i:i+window_size]
            Xs.append(window)
            ys.append(label)

    return np.array(Xs), np.array(ys)

# 5. 분리 로직 (핵심 변경 부분)
print(f"🔄 분리 방식: {SPLIT_METHOD.upper()} Split")

if SPLIT_METHOD == 'subject':
    # [방식 A] Subject-wise Split (기존 방식)
    test_subjects = [6, 11, 12] # 아까 찾은 우등생 조합

    train_df = df[~df['subject_id'].isin(test_subjects)].copy()
    test_df = df[df['subject_id'].isin(test_subjects)].copy()

    print(f"   Train Subjects: {train_df['subject_id'].unique()}")
    print(f"   Test Subjects : {test_df['subject_id'].unique()}")

    print("⏳ 윈도우 자르기 시작...")
    X_train, y_train = create_dataset(train_df, WINDOW_SIZE, STEP)
    X_test, y_test = create_dataset(test_df, WINDOW_SIZE, STEP)

elif SPLIT_METHOD == 'random':
    # [방식 B] Random Split (전체 섞기)
    print("⏳ 전체 데이터를 한 번에 윈도우로 변환 중...")

    # 1. 전체 데이터를 통째로 자름
    X_all, y_all = create_dataset(df, WINDOW_SIZE, STEP)

    # 2. 무작위로 섞어서 8:2로 나눔 (stratify=y_all로 클래스 비율 유지)
    print("🎲 데이터를 무작위로 섞어서 나누는 중 (Test Size: 20%)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all,
        test_size=0.2,
        random_state=42,
        shuffle=True,
        stratify=y_all
    )

print("-" * 30)
print(f"✅ 학습 데이터: {X_train.shape}")
print(f"✅ 평가 데이터: {X_test.shape}")
print("-" * 30)

# 6. 저장 (파일 이름은 그대로 둬야 train.py가 읽습니다)
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
print("💾 전처리 및 저장 완료!")

Classes: ['climbingdown' 'climbingup' 'jumping' 'lying' 'running' 'sitting'
 'standing' 'walking']
🔄 분리 방식: RANDOM Split
⏳ 전체 데이터를 한 번에 윈도우로 변환 중...
🎲 데이터를 무작위로 섞어서 나누는 중 (Test Size: 20%)...
------------------------------
✅ 학습 데이터: (53099, 100, 42)
✅ 평가 데이터: (13275, 100, 42)
------------------------------
💾 전처리 및 저장 완료!


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# 1. PKL 로드
df = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/HAR_data/RealWorld2016/RealWorld2016_Master_Sync.pkl")

WINDOW_SIZE = 100  # 2초 (50Hz)
STEP = 50          # 50% 겹침
FEATURE_COLS = [c for c in df.columns if 'acc_' in c or 'gyr_' in c] # 42개 컬럼

# 라벨 인코딩
le = LabelEncoder()
df['label'] = le.fit_transform(df['activity'])
print("Classes:", le.classes_)

# Windowing 함수 (수정됨: Instance Norm 추가)
def create_dataset(df, window_size, step):
    Xs, ys = [], []
    grouped = df.groupby(['subject_id', 'activity', 'trial'])

    for _, group in grouped:
        data = group[FEATURE_COLS].values
        label = group['label'].values[0]

        # 슬라이딩 윈도우
        for i in range(0, len(data) - window_size, step):
            window = data[i:i+window_size]

            Xs.append(window)
            ys.append(label)

    return np.array(Xs), np.array(ys)

# Train / Test 분리
test_subjects = [6, 11, 12]
test_df = df[df['subject_id'].isin(test_subjects)].copy()
train_df = df[~df['subject_id'].isin(test_subjects)].copy() # ~는 반대(NOT)를 의미
print(f"Train Subjects: {train_df['subject_id'].unique()}")
print(f"Test Subjects : {test_df['subject_id'].unique()}")

print("⏳ 윈도우 자르기 시작... (조금 걸림)")
X_train, y_train = create_dataset(train_df, WINDOW_SIZE, STEP)
X_test, y_test = create_dataset(test_df, WINDOW_SIZE, STEP)

print(f"✅ 학습 데이터: {X_train.shape}")
print(f"✅ 평가 데이터: {X_test.shape}")

# 저장
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)
print("💾 전처리 및 저장 완료!")

Classes: ['climbingdown' 'climbingup' 'jumping' 'lying' 'running' 'sitting'
 'standing' 'walking']
Train Subjects: [ 1  2  3  4  5  7  8  9 10 13 14 15]
Test Subjects : [ 6 11 12]
⏳ 윈도우 자르기 시작... (조금 걸림)
✅ 학습 데이터: (53459, 100, 42)
✅ 평가 데이터: (12915, 100, 42)
💾 전처리 및 저장 완료!


In [3]:
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 755.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=13e8797e90a06c415853ca29f7a3a7532b4ecba95c796df84d206b2f4cf1ac17
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=aa493ddbcea480963791db15c538d525b95b0338856643e9640648d08666f083
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


In [4]:
!python train.py

[01/50] Train Loss: 0.853 | F1: 0.7980  |  Test F1: 0.8650 (Best: 0.8650) Saved Best Model
[02/50] Train Loss: 0.551 | F1: 0.9340  |  Test F1: 0.9393 (Best: 0.9393) Saved Best Model
[03/50] Train Loss: 0.495 | F1: 0.9513  |  Test F1: 0.9525 (Best: 0.9525) Saved Best Model
[04/50] Train Loss: 0.469 | F1: 0.9569  |  Test F1: 0.9455 (Best: 0.9525) 
[05/50] Train Loss: 0.450 | F1: 0.9614  |  Test F1: 0.9639 (Best: 0.9639) Saved Best Model
[06/50] Train Loss: 0.437 | F1: 0.9635  |  Test F1: 0.9583 (Best: 0.9639) 
[07/50] Train Loss: 0.426 | F1: 0.9672  |  Test F1: 0.9704 (Best: 0.9704) Saved Best Model
[08/50] Train Loss: 0.419 | F1: 0.9681  |  Test F1: 0.9736 (Best: 0.9736) Saved Best Model
[09/50] Train Loss: 0.408 | F1: 0.9712  |  Test F1: 0.9679 (Best: 0.9736) 
[10/50] Train Loss: 0.402 | F1: 0.9716  |  Test F1: 0.9740 (Best: 0.9740) Saved Best Model
[11/50] Train Loss: 0.397 | F1: 0.9732  |  Test F1: 0.9693 (Best: 0.9740) 
[12/50] Train Loss: 0.393 | F1: 0.9749  |  Test F1: 0.9745 (Bes

In [5]:
!python visualize.py

--------------------------------------------------------------------------------
Running Visualization...
Loading weights from: /content/drive/MyDrive/Colab Notebooks/ASF/RealWorld2016/RealWorld2016_ASF.pth
--------------------------------------------------------------------------------
✅ Model weights loaded successfully!
--------------------------------------------------------------------------------
Evaluating Model Efficiency...
1. Parameters       : 0.1015 M
2. FLOPs / sample   : 5.825 M
3. Infer time       : 2.12 ms/sample
--------------------------------------------------------------------------------
Extracting features and predictions...
Generating Confusion Matrix...

Classification Report
              precision    recall  f1-score   support

climbingdown     0.9946    0.9867    0.9907      1507
  climbingup     0.9838    0.9866    0.9852      1785
     jumping     0.9962    0.9630    0.9793       270
       lying     0.9953    0.9927    0.9940      1924
     running     0.9

*5Fold & LOSO 탐색*



In [ ]:
import os
import copy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from tqdm import tqdm

# 기존에 작성하신 모델 파일들 불러오기
from model_utils import set_seed, seed_worker, ASFDCLClassifier

# ------------------------------------------------------------------------------
# 1. 설정 및 데이터 로드 (전체 데이터를 메모리에 올림)
# ------------------------------------------------------------------------------
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/HAR_data/RealWorld2016/RealWorld2016_Master_Sync.pkl"
WINDOW_SIZE = 100
STEP = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_and_preprocess_all():
    print("⏳ 전체 데이터 로드 및 전처리 중...")
    df = pd.read_pickle(DATA_PATH)

    # 컬럼 선택
    feature_cols = [c for c in df.columns if 'acc_' in c or 'gyr_' in c]

    # 라벨 인코딩
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['activity'])

    # 윈도우 자르기 (정규화 없이 Raw로)
    Xs, ys, groups = [], [], []

    # 그룹화 (사람 단위로 쪼개기 위해)
    grouped = df.groupby(['subject_id', 'activity', 'trial'])

    for _, group in grouped:
        data = group[feature_cols].values
        label = group['label'].values[0]
        subj_id = group['subject_id'].values[0]

        for i in range(0, len(data) - WINDOW_SIZE, STEP):
            window = data[i:i+WINDOW_SIZE]
            Xs.append(window)
            ys.append(label)
            groups.append(subj_id) # 나중에 사람 기준으로 나누기 위해 저장

    Xs = np.array(Xs).astype(np.float32)
    ys = np.array(ys).astype(np.longlong)
    groups = np.array(groups)


    print(f"✅ 전체 데이터 준비 완료: {Xs.shape}")
    return Xs, ys, groups

# ------------------------------------------------------------------------------
# 2. 간단한 Dataset 클래스
# ------------------------------------------------------------------------------
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx])

# ------------------------------------------------------------------------------
# 3. 학습 함수 (1 Epoch만 빠르게 돌려서 간 보기 or Full 학습)
# ------------------------------------------------------------------------------
def train_and_evaluate(train_idx, test_idx, X, y, fold_num, epochs=10):
    # 데이터 분할
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # 데이터셋 & 로더
    train_ds = SimpleDataset(X_train, y_train)
    test_ds = SimpleDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=0)

    # 모델 초기화 (매 폴드마다 새로 만들어야 함)
    model = ASFDCLClassifier(
        input_channels=42, latent_dim=64, hidden_dim=64,
        num_classes=8, num_heads=4, projection_dim=128
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    # 학습 루프
    best_f1 = 0.0

    print(f"\n🚀 Fold {fold_num} 학습 시작 (Epochs: {epochs})...")
    for epoch in range(epochs):
        model.train()
        for bx, by in train_loader:
            bx, by = bx.to(DEVICE), by.to(DEVICE)
            optimizer.zero_grad()
            logits = model(bx) # 단순 분류 로스만 사용하여 빠르게 탐색 (필요시 ASF 로스 사용)
            loss = criterion(logits, by)
            loss.backward()
            optimizer.step()

        # 평가
        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for bx, by in test_loader:
                bx, by = bx.to(DEVICE), by.to(DEVICE)
                logits = model(bx)
                preds = torch.argmax(logits, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(by.cpu().numpy())

        f1 = f1_score(all_labels, all_preds, average='macro')
        if f1 > best_f1:
            best_f1 = f1

    return best_f1

# ------------------------------------------------------------------------------
# 4. 메인: K-Fold 서치
# ------------------------------------------------------------------------------
def main():
    set_seed(42)

    # 1. 데이터 로드
    X, y, groups = load_and_preprocess_all()

    # 2. Group K-Fold 설정 (5등분 -> 15명이니까 그룹당 3명씩 테스트)
    gkf = GroupKFold(n_splits=5)

    results = []

    # 3. 폴드별 순회
    for fold, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups)):
        # 현재 테스트에 쓰이는 사람 ID 확인
        test_subjects = np.unique(groups[test_idx])
        train_subjects = np.unique(groups[train_idx])

        print("="*60)
        print(f"📌 Fold {fold+1}/5")
        print(f"   ▶ Test Subjects : {test_subjects}") # 누구를 테스트하는가?
        print(f"   ▶ Train Subjects: {len(train_subjects)}명")

        # 학습 수행 (여기서 Epoch을 조절하세요. 찾기용이니 5~10 정도로 짧게)
        score = train_and_evaluate(train_idx, test_idx, X, y, fold_num=fold+1, epochs=10)

        print(f"   🏆 Best F1 Score: {score:.4f}")
        results.append({
            "fold": fold+1,
            "test_subjects": test_subjects.tolist(),
            "f1_score": score
        })

    # 4. 결과 종합
    print("\n" + "="*60)
    print("📊 최종 결과 리포트 (점수 높은 순)")
    print("="*60)

    # 점수 높은 순으로 정렬
    sorted_results = sorted(results, key=lambda x: x['f1_score'], reverse=True)

    for rank, res in enumerate(sorted_results):
        print(f"{rank+1}위 | Test Subjects: {res['test_subjects']} | F1: {res['f1_score']:.4f}")

    print(f"\n💡 전체 평균 F1 Score: {np.mean([r['f1_score'] for r in results]):.4f}")

if __name__ == "__main__":
    main()

⏳ 전체 데이터 로드 및 전처리 중...
✅ 전체 데이터 준비 완료: (66374, 100, 42)
📌 Fold 1/5
   ▶ Test Subjects : [ 2  5 13]
   ▶ Train Subjects: 12명

🚀 Fold 1 학습 시작 (Epochs: 10)...
   🏆 Best F1 Score: 0.7223
📌 Fold 2/5
   ▶ Test Subjects : [ 8 10 11]
   ▶ Train Subjects: 12명

🚀 Fold 2 학습 시작 (Epochs: 10)...
   🏆 Best F1 Score: 0.6687
📌 Fold 3/5
   ▶ Test Subjects : [1 6 7]
   ▶ Train Subjects: 12명

🚀 Fold 3 학습 시작 (Epochs: 10)...
   🏆 Best F1 Score: 0.7080
📌 Fold 4/5
   ▶ Test Subjects : [ 3 12 15]
   ▶ Train Subjects: 12명

🚀 Fold 4 학습 시작 (Epochs: 10)...
   🏆 Best F1 Score: 0.6792
📌 Fold 5/5
   ▶ Test Subjects : [ 4  9 14]
   ▶ Train Subjects: 12명

🚀 Fold 5 학습 시작 (Epochs: 10)...
   🏆 Best F1 Score: 0.7909

📊 최종 결과 리포트 (점수 높은 순)
1위 | Test Subjects: [4, 9, 14] | F1: 0.7909
2위 | Test Subjects: [2, 5, 13] | F1: 0.7223
3위 | Test Subjects: [1, 6, 7] | F1: 0.7080
4위 | Test Subjects: [3, 12, 15] | F1: 0.6792
5위 | Test Subjects: [8, 10, 11] | F1: 0.6687

💡 전체 평균 F1 Score: 0.7138


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import LeaveOneGroupOut # 이 함수가 핵심입니다
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from model_utils import set_seed, ASFDCLClassifier  # 기존 파일 활용

# 설정
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/HAR_data/RealWorld2016/RealWorld2016_Master_Sync.pkl"
WINDOW_SIZE = 100
STEP = 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_and_preprocess_all():
    print("⏳ 전체 데이터 로드 및 전처리 중...")
    df = pd.read_pickle(DATA_PATH)

    # 컬럼 선택
    feature_cols = [c for c in df.columns if 'acc_' in c or 'gyr_' in c]

    # 라벨 인코딩
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['activity'])

    # 윈도우 자르기 (정규화 없이 Raw로)
    Xs, ys, groups = [], [], []

    # 그룹화 (사람 단위로 쪼개기 위해)
    grouped = df.groupby(['subject_id', 'activity', 'trial'])

    for _, group in grouped:
        data = group[feature_cols].values
        label = group['label'].values[0]
        subj_id = group['subject_id'].values[0]

        for i in range(0, len(data) - WINDOW_SIZE, STEP):
            window = data[i:i+WINDOW_SIZE]
            Xs.append(window)
            ys.append(label)
            groups.append(subj_id) # 나중에 사람 기준으로 나누기 위해 저장

    Xs = np.array(Xs).astype(np.float32)
    ys = np.array(ys).astype(np.longlong)
    groups = np.array(groups)


    print(f"✅ 전체 데이터 준비 완료: {Xs.shape}")
    return Xs, ys, groups

# ------------------------------------------------------------------------------
# 2. 간단한 Dataset 클래스
# ------------------------------------------------------------------------------
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.tensor(self.y[idx])

def train_and_evaluate(train_idx, test_idx, X, y, subject_id, epochs=10):
    # 데이터 분할
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Dataset & Loader
    train_ds = SimpleDataset(X_train, y_train)
    test_ds = SimpleDataset(X_test, y_test)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)

    # 모델 초기화
    model = ASFDCLClassifier(
        input_channels=42, latent_dim=64, hidden_dim=64,
        num_classes=8, num_heads=4, projection_dim=128
    ).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    best_f1 = 0.0

    # 짧은 학습
    for epoch in range(epochs):
        model.train()
        for bx, by in train_loader:
            bx, by = bx.to(DEVICE), by.to(DEVICE)
            optimizer.zero_grad()
            output = model(bx)
            loss = criterion(output, by)
            loss.backward()
            optimizer.step()

        # 평가
        model.eval()
        preds, labels = [], []
        with torch.no_grad():
            for bx, by in test_loader:
                bx, by = bx.to(DEVICE), by.to(DEVICE)
                output = model(bx)
                preds.extend(torch.argmax(output, dim=1).cpu().numpy())
                labels.extend(by.cpu().numpy())

        current_f1 = f1_score(labels, preds, average='macro')
        if current_f1 > best_f1:
            best_f1 = current_f1

    return best_f1

def main():
    set_seed(42)
    X, y, groups = load_and_preprocess_all() # 이전 코드의 함수 사용

    # 🔥 핵심 변경: LeaveOneGroupOut (총 15번 수행)
    logo = LeaveOneGroupOut()

    results = []

    print(f"\n🚀 Leave-One-Subject-Out (총 15회) 시작...\n")

    for fold, (train_idx, test_idx) in enumerate(logo.split(X, y, groups)):
        test_subject = np.unique(groups[test_idx])[0] # 현재 테스트하는 1명

        print(f"[{fold+1}/15] Test Subject: {test_subject}번 평가 중...")

        score = train_and_evaluate(train_idx, test_idx, X, y, test_subject, epochs=5) # 5 Epoch만 빠르게

        print(f"   -> Best F1: {score:.4f}")
        results.append({"subject": test_subject, "f1": score})

    # 결과 리포트
    print("\n" + "="*40)
    print("📊 피험자별 난이도 리포트 (F1 낮은 순 = 어려운 사람)")
    print("="*40)

    # 점수가 낮은 순(어려운 사람)으로 정렬
    sorted_res = sorted(results, key=lambda x: x['f1'])

    for res in sorted_res:
        print(f"Subject {res['subject']}: {res['f1']:.4f}")

    print(f"\n⭐ 전체 평균 F1: {np.mean([r['f1'] for r in results]):.4f}")

if __name__ == "__main__":
    main() # 실행 전에 이전 코드의 load_and_preprocess_all, SimpleDataset 정의가 필요합니다.

⏳ 전체 데이터 로드 및 전처리 중...
✅ 전체 데이터 준비 완료: (66374, 100, 42)

🚀 Leave-One-Subject-Out (총 15회) 시작...

[1/15] Test Subject: 1번 평가 중...
   -> Best F1: 0.7379
[2/15] Test Subject: 2번 평가 중...
   -> Best F1: 0.6829
[3/15] Test Subject: 3번 평가 중...
   -> Best F1: 0.4576
[4/15] Test Subject: 4번 평가 중...
   -> Best F1: 0.7428
[5/15] Test Subject: 5번 평가 중...
   -> Best F1: 0.7914
[6/15] Test Subject: 6번 평가 중...
   -> Best F1: 0.8484
[7/15] Test Subject: 7번 평가 중...
   -> Best F1: 0.7624
[8/15] Test Subject: 8번 평가 중...
   -> Best F1: 0.4430
[9/15] Test Subject: 9번 평가 중...
   -> Best F1: 0.8313
[10/15] Test Subject: 10번 평가 중...
   -> Best F1: 0.7666
[11/15] Test Subject: 11번 평가 중...
   -> Best F1: 0.8542
[12/15] Test Subject: 12번 평가 중...
   -> Best F1: 0.9131
[13/15] Test Subject: 13번 평가 중...
   -> Best F1: 0.7589
[14/15] Test Subject: 14번 평가 중...
   -> Best F1: 0.7931
[15/15] Test Subject: 15번 평가 중...
   -> Best F1: 0.7184

📊 피험자별 난이도 리포트 (F1 낮은 순 = 어려운 사람)
Subject 8: 0.4430
Subject 3: 0.4576
Subject 2: 